this script describe the funding portfolio and the recieved funding portfolio of countries. 

In [1]:
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.spatial import distance

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv"
eufile_path='../../data/nf_folder/Data/AdditionalData/cntry_convert.xlsx'

In [3]:
fund_type_path="../../data/nf_folder/Data/DerivedData/Derived/fund_type/fund_type.csv"
recv_type_path="../../data/nf_folder/Data/DerivedData/Derived/fund_type/recv_type.csv"

In [4]:
pubs_author_funder = pd.read_csv(pubs_path)
pubs_author_funder['author_country'] = pubs_author_funder['author_country'].apply(literal_eval)
pubs_author_funder['funder_country'] = pubs_author_funder['funder_country'].apply(
    lambda x:literal_eval(x) if pd.notnull(x) else "Not-Funded")
pubs_author_funder.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [5]:
pubs_author_funder['author_distinct']=pubs_author_funder['author_country'].apply(lambda x:list(set(x)))
pubs_author_funder['funder_distinct']=pubs_author_funder['funder_country'].apply(
    lambda x:list(set(x)) if x!='Not-Funded' else 'Not-Funded')

In [6]:
pubs_filter=pubs_author_funder[pubs_author_funder.funder_distinct!='Not-Funded']
pubs_filter.head()

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct,funder_distinct
0,30507292,[Turkey],[Turkey],77,2009,0.485,6,[Turkey],[Turkey]
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9,"[France, United States]",[United States]
2,30574691,[Russia],[Russia],83,2009,0.312,4,[Russia],[Russia]
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1,[Spain],[Spain]
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3,"[Germany, Spain]",[Spain]


In [7]:
eu_df=pd.read_excel(eufile_path)
eu_df.head()

<ipython-input-7-0f652c5f6423>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  eu_df=pd.read_excel(eufile_path)


,Clarivate country,Country,Continent,EU
0,*EFFACER,NaN,NaN,NaN
1,_indeterminé,_indeterminé,NaN,NaN
2,AFGHANISTAN,Afghanistan,Asia,NaN
3,SOUTH-AFRICA,South Africa,Africa,NaN
4,ALBANIA,Albania,Europe,NaN


In [8]:
eulist=eu_df[eu_df.EU==1].Country.unique()
eulist

array(['Germany', 'United Kingdom', 'Austria', 'Belgium', 'Spain',
       'Estonia', 'Finland', 'France', 'Greece', 'Hungary', 'Netherlands',
       'Poland', 'Portugal', 'Czech Republic', 'Romania', 'Slovakia',
       'Slovenia', 'Sweden', 'Bulgaria', 'Cyprus', 'Croatia', 'Denmark',
       'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta'],
      dtype=object)

In [6]:
fund_type_frac=[]
for index, row in pubs_filter.iterrows():
    year=row['year']
    funderlist=row['funder_distinct']
    authorlist=row['author_distinct']
    n_funder=len(funderlist)
    for f in funderlist:
        if (f in authorlist) and len(authorlist)>1:
            type='coll'
        elif (f in authorlist) and len(authorlist)==1:
            type='dome'
        else:
            type='fore'
        fund_type_frac.append([f,year,1/n_funder,type])

In [56]:
pubs_author_funder[30:40]

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct,funder_distinct
30,30714112,"[United States, United States]","[United States, United States]",89,2009,0.957,7,[United States],[United States]
31,30719103,[Germany],[Germany],88,2009,4.545,60,[Germany],[Germany]
32,30720927,"[Romania, Malaysia]",[Malaysia],88,2009,2.803,37,"[Romania, Malaysia]",[Malaysia]
33,30720928,"[Italy, Italy]",[Italy],88,2009,0.455,6,[Italy],[Italy]
34,30721286,"[United Kingdom, India]",[EU],1,2009,4.064,69,"[United Kingdom, India]",[EU]
35,30721851,"[Italy, Bulgaria, Romania, India]",[EU],99,2009,1.664,23,"[Romania, Italy, India, Bulgaria]",[EU]
36,30723391,[Brazil],"[Brazil, Brazil, Brazil]",1,2009,3.475,59,[Brazil],[Brazil]
37,30723714,[Spain],"[Spain, EU, Spain, Spain]",93,2009,0.193,3,[Spain],"[Spain, EU]"
38,30723716,[India],[India],93,2009,0.898,14,[India],[India]
39,30723717,"[South Korea, South Korea]",[South Korea],93,2009,0.321,5,[South Korea],[South Korea]


In [65]:
recv_type=[]
for index, row in pubs_author_funder.iterrows():
    year=row['year']
    funderlist=row['funder_distinct']
    authorlist=row['author_distinct']
    n_author=len(authorlist)
    for a in authorlist:
        if funderlist=='Not-Funded':
            type='not-funded'
        elif (a in eulist)&("EU" in funderlist):
            type='coll'
        elif (a in funderlist)&(len(funderlist)>1):
            type='coll'
        elif (a in funderlist)&(len(funderlist)==1):
            type='dome'
        elif (a not in funderlist):
            type='fore'
        recv_type.append([a,year,1/n_author,type])
    

In [27]:
def get_topn(datalist,n):
    df=pd.DataFrame(datalist,columns=['cntry','year','count','type'])
    df=df[df.cntry!='EU']
    cntry_df=df.groupby(['cntry'])['count'].sum().reset_index().sort_values(
        by='count',ascending=False)
    topnlist=cntry_df['cntry'][:n].values
    return topnlist

In [20]:
def norm(datalist):
    data_df=pd.DataFrame(datalist,columns=['cntry','year','count','type'])
    data_df=data_df.groupby(['cntry','type'])['count'].sum().reset_index(name='count')
    data_df=data_df.pivot_table(index='cntry',columns='type',values='count',fill_value=0)
    data_df=data_df.div(data_df.sum(axis=1), axis=0)
    return data_df
    

In [35]:
fund_type_df=norm(fund_type_frac)
fund_type_df.to_csv(fund_type_path)

In [66]:
recv_type_df=norm(recv_type)
recv_type_df.to_csv(recv_type_path)
recv_type_df.head()

type,coll,dome,fore,not-funded
cntry,,,,
Afghanistan,0.000000,0.002975,0.385776,0.611249
Albania,0.003129,0.007927,0.153920,0.835023
Algeria,0.015532,0.067781,0.086895,0.829792
Andorra,0.034355,0.085887,0.395432,0.484326
Angola,0.071026,0.009717,0.449565,0.469692


In [28]:
topnlist=get_topn(fund_type_frac,20)
topnlist

array(['China', 'United States', 'Japan', 'Germany', 'South Korea',
       'United Kingdom', 'Canada', 'Spain', 'India', 'Brazil',
       'Australia', 'France', 'Russia', 'Taiwan', 'Italy', 'Switzerland',
       'Sweden', 'Poland', 'Iran', 'Netherlands'], dtype=object)